In [34]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
...<article class="day-desc"><h2>--- Day 13: Mine Cart Madness ---</h2><p>A crop of this size requires significant logistics to transport produce, soil, fertilizer, and so on. The Elves are very busy pushing things around in <em>carts</em> on some kind of rudimentary system of tracks they've come up with.</p>
<p>Seeing as how cart-and-track systems don't appear in recorded history for <span title="Time anomalies! How do they work?!">another 1000 years</span>, the Elves seem to be making this up as they go along. They haven't even figured out how to avoid collisions yet.</p>
<p>You map out the tracks (your puzzle input) and see where you can help.</p>
<p>Tracks consist of straight paths (<code>|</code> and <code>-</code>), curves (<code>/</code> and <code>\</code>), and intersections (<code>+</code>). Curves connect exactly two perpendicular pieces of track; for example, this is a closed loop:</p>
<pre><code>/----\
|    |
|    |
\----/
</code></pre>
<p>Intersections occur when two perpendicular paths cross. At an intersection, a cart is capable of turning left, turning right, or continuing straight.  Here are two loops connected by two intersections:</p>
<pre><code>/-----\
|     |
|  /--+--\
|  |  |  |
\--+--/  |
   |     |
   \-----/
</code></pre>

<p>Several <em>carts</em> are also on the tracks. Carts always face either up (<code>^</code>), down (<code>v</code>), left (<code>&lt;</code>), or right (<code>&gt;</code>). (On your initial map, the track under each cart is a straight path matching the direction the cart is facing.)</p>
<p>Each time a cart has the option to turn (by arriving at any intersection), it turns <em>left</em> the first time, goes <em>straight</em> the second time, turns <em>right</em> the third time, and then repeats those directions starting again with <em>left</em> the fourth time, <em>straight</em> the fifth time, and so on. This process is independent of the particular intersection at which the cart has arrived - that is, the cart has no per-intersection memory.</p>
<p>Carts all move at the same speed; they take turns moving a single step at a time. They do this based on their <em>current location</em>: carts on the top row move first (acting from left to right), then carts on the second row move (again from left to right), then carts on the third row, and so on.  Once each cart has moved one step, the process repeats; each of these loops is called a <em>tick</em>.</p>
<p>For example, suppose there are two carts on a straight track:</p>
<pre><code>|  |  |  |  |
v  |  |  |  |
|  v  v  |  |
|  |  |  v  X
|  |  ^  ^  |
^  ^  |  |  |
|  |  |  |  |
</code></pre>
<p>First, the top cart moves. It is facing down (<code>v</code>), so it moves down one square.  Second, the bottom cart moves.  It is facing up (<code>^</code>), so it moves up one square. Because all carts have moved, the first tick ends.  Then, the process repeats, starting with the first cart.  The first cart moves down, then the second cart moves up - right into the first cart, colliding with it! (The location of the crash is marked with an <code>X</code>.) This ends the second and last tick.</p><p>
</p><p>Here is a longer example:</p>
<pre><code>/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   
<pre><code>
</code></pre>/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   
<pre><code>
</code></pre>/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-<--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  v----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \<+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----\
| /-+--v-\  |
| | |  | |  |
\-+-/  ^-+--/
  \------/   
<pre><code>
</code></pre>/---\        
|   |  /----\
| /-+--+-\  |
| | |  X |  |
\-+-/  \-+--/
  \------/   
</code></pre>
<p>After following their respective paths for a while, the carts eventually crash.  To help prevent crashes, you'd like to know <em>the location of the first crash</em>. Locations are given in <code>X,Y</code> coordinates, where the furthest left column is <code>X=0</code> and the furthest top row is <code>Y=0</code>:</p>
<pre><code>           111
 0123456789012
0/---\        
1|   |  /----\
2| /-+--+-\  |
3| | |  X |  |
4\-+-/  \-+--/
5  \------/   
</code></pre>
<p>In this example, the location of the first crash is <code><em>7,3</em></code>.</p>
</article>


In [61]:
from tabulate import tabulate


example = r"""
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/
"""


def print_map(map: list[list[str]]) -> None:
    for line in map:
        print("".join(line))


def parse(map: str) -> list[list[str]]:
    return [list(l) for l in map.splitlines() if l]


def find_carts(map: list[list[str]]) -> dict[tuple[int, int], str, str]:
    carts = {}
    for r, line in enumerate(map):
        for c, ch in enumerate(line):
            if ch == "^":
                map[r][c] = "|"
                carts[(r, c)] = -1, 0, 0
            elif ch == ">":
                map[r][c] = "-"
                carts[(r, c)] = 0, 1, 0
            elif ch == "v":
                map[r][c] = "|"
                carts[(r, c)] = 1, 0, 0
            elif ch == "<":
                map[r][c] = "-"
                carts[(r, c)] = 0, -1, 0
    return carts


def turn_left(dr: int, dc: int) -> tuple[int, int]:
    if (dr, dc) == (-1, 0):  # up
        return 0, -1  # left
    if (dr, dc) == (1, 0):  # down
        return 0, 1  # right
    if (dr, dc) == (0, 1):  # right
        return -1, 0  # up
    return 1, 0  # left -> down


def turn_right(dr: int, dc: int) -> tuple[int, int]:
    dr, dc = turn_left(dr, dc)
    return -dr, -dc


def new_direction(map, row, col, dr, dc, turn):
    if map[row][col] == "+":
        if turn == 0:  # turn left
            return *turn_left(dr, dc), 1
        elif turn == 1:  # straight
            return dr, dc, 2
        else:  # turn == 2 = right
            return *turn_right(dr, dc), 0
    if map[row][col] == "/":
        return -dc, -dr, turn
    if map[row][col] == "\\":
        return dc, dr, turn
    return dr, dc, turn


def first_crash(map_str: str) -> tuple[int, int]:
    map = parse(map_str)
    carts = find_carts(map)
    while True:
        for (row, col), (dr, dc, turn) in sorted(carts.items()):
            dr, dc, turn = new_direction(map, row, col, dr, dc, turn)

            r, c = row + dr, col + dc
            if (r, c) in carts:
                return c, r

            del carts[(row, col)]
            carts[(r, c)] = dr, dc, turn


print(f"Example: {','.join(str(i) for i in first_crash(example))} should be 7,3")

Example: 7,3 should be 7,3


In [62]:
with open("../input/day13.txt") as f:
    puzzle = f.read()

print(f"Part I: {','.join((str(i) for i in first_crash(puzzle)))}")

Part I: 76,108


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>76,108</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>There isn't much you can do to prevent crashes in this ridiculous system. However, by predicting the crashes, the Elves know where to be in advance and <em>instantly remove the two crashing carts</em> the moment any crash occurs.</p>
<p>They can proceed like this for a while, but eventually, they're going to run out of carts. It could be useful to figure out where the last cart that <em>hasn't</em> crashed will end up.</p>
<p>For example:</p>
<pre><code>/&gt;-&lt;\  
|   |  
| /&lt;+-\
| | | v
\&gt;+&lt;/ |
  |   ^
  \&lt;-&gt;/
<pre><code>
</code></pre>/---\  
|   |  
| v-+-\
| | | |
\-+-/ |
  |   |
  ^---^
<pre><code>
</code></pre>/---\  
|   |  
| /-+-\
| v | |
\-+-/ |
  ^   ^
  \---/
<pre><code>
</code></pre>/---\  
|   |  
| /-+-\
| | | |
\-+-/ <em>^</em>
  |   |
  \---/
</code></pre>
<p>After four very expensive crashes, a tick ends with only one cart remaining; its final location is <code><em>6,4</em></code>.</p>
<p><em>What is the location of the last cart</em> at the end of the first tick where it is the only cart left?</p>
</article>

</main>


In [63]:
from more_itertools import first


example_II = r"""
/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/
"""


def last_cart(map_str: str) -> tuple[int, int]:
    map = parse(map_str)
    carts = find_carts(map)
    while True:
        active_carts = sorted(carts.items())
        if len(active_carts) == 1:
            (row, col), _ = first(active_carts)
            return col, row

        for (row, col), (dr, dc, turn) in active_carts:
            if (row, col) not in carts:
                continue

            dr, dc, turn = new_direction(map, row, col, dr, dc, turn)

            r, c = row + dr, col + dc
            if (r, c) in carts:
                del carts[(r, c)]
            else:
                carts[(r, c)] = dr, dc, turn
            del carts[(row, col)]


print(f"Example: {','.join(str(i) for i in last_cart(example_II))} should be 6,4")

Example: 6,4 should be 6,4


In [64]:
print(f"Part II: {','.join((str(i) for i in last_cart(puzzle)))}")

Part II: 2,84


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2,84</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
